In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
from cartopy import crs as ccrs, feature as feature
import matplotlib.colors as colors
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from shapely.geometry.polygon import LinearRing
import matplotlib.ticker as mticker

In [ ]:
# Load catches
fname = 'data/hump_filled_RR.csv'
df = pd.read_csv(fname, sep= ';')
# Select higher latitudes and class
df = df[df['lat'] <= -40]
df = df[df['class'] < 7] # 13576 records
#df = df[df['class'] == 1] # 13208 records

In [ ]:
# create labels for decades
labels = ['{0}-{1}'.format(i, i + 9) for i in range(1900, 1961, 10)]
#df = df[(df['year'] >= 1900) & (df['year'] < 1910)]
print(labels)
df['decade'] = pd.cut(df['year'], range(1900, 1971, 10), right=True, labels=labels)

In [ ]:
# extract the months and the decade
NOV = df[df['month'] == 11]
DEC = df[df['month'] == 12]
JAN = df[df['month'] == 1]
FEB = df[df['month'] == 2]
N30 = NOV[NOV['decade']=='1930-1939']
D30 = DEC[DEC['decade']=='1930-1939']
J30 = JAN[JAN['decade']=='1930-1939']
F30 = FEB[FEB['decade']=='1930-1939']
total = len(N30)+len(D30)+len(J30)+len(F30)

In [ ]:
# Generate Figure 1a-d
fig, axs = plt.subplots(2, 2,sharey=True, tight_layout=True)
axs[0,0].hist(NOV['year'], bins= np.arange(1900,1961,10)-0.5, color='green', zorder=2, rwidth=0.8, align = "left") ## 10 for Decadal or 1 for annual
axs[0,0].set_title('a) November', loc = 'left', fontsize=12)
axs[0,0].grid(True, which='major')
axs[0,1].hist(DEC['year'], bins=np.arange(1900,1961,10)-0.5, color='green', zorder=2, rwidth=0.8, align = "left")  ## 10 for Decadal
axs[0,1].set_title('b) December', loc = 'left', fontsize=12)
axs[0,1].grid(True)
axs[1,0].hist(JAN['year'], bins=np.arange(1900,1961,10)-0.5, color='green', zorder=2, rwidth=0.8, align = "left")  ## 10 for Decadal
axs[1,0].set_title('c) January', loc = 'left', fontsize=12)
axs[1,0].grid(True)
axs[1,1].hist(FEB['year'], bins=np.arange(1900,1961,10)-0.5, color='green', zorder=2, rwidth=0.8, align = "left")  ## 10 for Decadal
axs[1,1].set_title('d) February', loc = 'left', fontsize=12)
axs[1,1].grid(True)

plt.yticks(np.arange(0, 12000, 2000,)) ##or 2000 for annual ##7000 for decadal

x = range(1905, 1976, 10) 
cc= dict.fromkeys(x)

axs[0,0].set(ylabel='Whale Catches')
axs[1,0].set(ylabel='Whale Catches')


for ax in axs.flat:
    ax.set_xticks(range(1900,1961,10))
    #ax.set_xticklabels(cc.keys())
    ax.xaxis.get_label().set_fontsize(12)
    ax.yaxis.get_label().set_fontsize(12)
    ax.xaxis.set_tick_params(labelsize=8)
    ax.yaxis.set_tick_params(labelsize=12)
    ax.set_xlim([1890, 1970])
    ax.set_ylim([0, 11000])  #6500 for Decadal or 2000 for annual
#for ax in axs.flat:
#    ax.label_outer()    
plt.savefig('figures/Fig1a-d.png',dpi=300)

In [ ]:
# Generate Figure 1e (to be combined graphically)
fig = plt.figure(figsize= (15, 7))
ax1 = plt.subplot(1,1,1, projection=ccrs.Miller())

ax1.set_extent([-120, 180, -30, -75], crs=ccrs.PlateCarree())

lons = [-70, -70, 150, 150]
lats = [-40, -70, -70, -40]
ring = LinearRing(list(zip(lons, lats)))
ax1.add_geometries([ring], ccrs.PlateCarree(), facecolor='none', edgecolor='purple', linewidth = 1)

ax1.set_xlabel('Longitude', labelpad=2, fontsize = 14)
ax1.set_ylabel('Latitude', labelpad=2, fontsize = 14)
ax1.coastlines() 

gl1 = ax1.gridlines(linewidth = 0.5, color = 'grey', alpha = 0.3, linestyle = '-', draw_labels=True)
gl1.xlocator = mticker.MultipleLocator(60)
gl1.ylocator = mticker.MultipleLocator(10)

gl1.top_labels = False
gl1.right_labels = False

gl1.xformatter = LONGITUDE_FORMATTER
gl1.yformatter = LATITUDE_FORMATTER
gl1.xlabel_style = {'size': 13, 'color': 'black'}
gl1.ylabel_style = {'size': 13, 'color': 'black'}

MS = 35

ax1.scatter(N30['long'], N30['lat'], MS, marker='.', facecolor = 'goldenrod', 
            transform=ccrs.PlateCarree(), zorder=100, label = 'November')
ax1.scatter(D30['long'], D30['lat'], MS, marker='.', facecolor = 'cornflowerblue', 
            transform=ccrs.PlateCarree(), zorder=100, label = 'December')
ax1.scatter(J30['long'], J30['lat'], MS, marker='.', facecolor = 'orangered', 
            transform=ccrs.PlateCarree(), zorder=100, label = 'January')
ax1.scatter(F30['long'], F30['lat'], MS, marker='.', facecolor = 'limegreen', 
            transform=ccrs.PlateCarree(), zorder=100, label = 'February')
ax1.legend(loc='upper left', fontsize=14)
ax1.set_title('e) 1930-39', loc='left', fontsize=14)
plt.tight_layout
plt.savefig('figures/Fig1e.png',dpi=300)